In [ ]:
# #https://huggingface.co/docs/transformers/v4.29.1/en/model_doc/bert#transformers.BertForSequenceClassification
# import torch
# from transformers import AutoTokenizer, BertForSequenceClassification

# tokenizer = AutoTokenizer.from_pretrained("textattack/bert-base-uncased-yelp-polarity")
# model = BertForSequenceClassification.from_pretrained("textattack/bert-base-uncased-yelp-polarity", problem_type="multi_label_classification")

# inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")

# with torch.no_grad():
#     logits = model(**inputs).logits

# predicted_class_ids = torch.arange(0, logits.shape[-1])[torch.sigmoid(logits).squeeze(dim=0) > 0.5]

# # To train a model on `num_labels` classes, you can pass `num_labels=num_labels` to `.from_pretrained(...)`
# num_labels = len(model.config.id2label)
# model = BertForSequenceClassification.from_pretrained(
#     "textattack/bert-base-uncased-yelp-polarity", num_labels=num_labels, problem_type="multi_label_classification"
# )

# labels = torch.sum(
#     torch.nn.functional.one_hot(predicted_class_ids[None, :].clone(), num_classes=num_labels), dim=1
# ).to(torch.float)
# loss = model(**inputs, labels=labels).loss

In [ ]:
# from datasets import load_dataset

# #dataset = load_dataset("yelp_review_full")
# dataset = load_dataset("amazon_reviews_multi", "en", "train")
# dataset["train"][100]

In [6]:
from datasets import load_dataset

dataset = load_dataset("amazon_reviews_multi", "en", "train")




Extracting data files: 100%|███████████████████| 1/1 [00:00<00:00, 242.03it/s]


Extracting data files: 100%|███████████████████| 1/1 [00:00<00:00, 291.80it/s]


Extracting data files: 100%|███████████████████| 1/1 [00:00<00:00, 318.45it/s]

Generating train split:   4%| | 7025/200000 [00:00<00:05, 35721.88 examples/s]
Generating train split:   6%| | 11000/200000 [00:00<00:05, 37217.07 examples/s
Generating train split:   7%| | 14942/200000 [00:00<00:04, 38076.01 examples/s
Generating train split:   9%| | 18904/200000 [00:00<00:04, 38629.43 examples/s
Generating train split:  11%| | 22816/200000 [00:00<00:04, 38793.79 examples/s
Generating train split:  13%|▏| 26762/200000 [00:00<00:04, 39008.39 examples/s
Generating train split:  16%|▏| 32698/200000 [00:00<00:04, 39240.54 examples/s
Generating train split:  19%|▏| 38566/200000 [00:01<00:04, 39191.69 examples/s
Generating train split:  22%|▏| 44443/200000 [00:01<00:03, 39183.56 examples/s
Generating train split:  24%|▏| 48378/200000 

Dataset amazon_reviews_multi downloaded and prepared to /Users/apmcm/.cache/huggingface/datasets/amazon_reviews_multi/en-data_dir=train/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609. Subsequent calls will reuse this data.


100%|██████████████████████████████████████████| 3/3 [00:00<00:00, 838.47it/s]


{'review_id': 'en_0143676',
 'product_id': 'product_en_0610451',
 'reviewer_id': 'reviewer_en_0377453',
 'stars': 1,
 'review_body': 'Two nights in the water tide to our dock in the lake..... I’d say something liked this.',
 'review_title': 'Let the picture tell you how go this is',
 'language': 'en',
 'product_category': 'sports'}

In [9]:
import pprint

In [10]:
pprint.pprint(dataset["train"][100])

{'language': 'en',
 'product_category': 'sports',
 'product_id': 'product_en_0610451',
 'review_body': 'Two nights in the water tide to our dock in the lake..... I’d '
                'say something liked this.',
 'review_id': 'en_0143676',
 'review_title': 'Let the picture tell you how go this is',
 'reviewer_id': 'reviewer_en_0377453',
 'stars': 1}


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["text"])

# small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
# small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [ ]:
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")

small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(10))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(10))

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=8)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=8)


from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5) # 5 stars


from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)


from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
        



In [ ]:
import evaluate

metric = evaluate.load("accuracy")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

In [ ]:
small_train_dataset.shape